In [1]:
import torch
from pytorch_lightning import LightningModule, Trainer
from torch import nn
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchmetrics import Accuracy
from torchvision import transforms
from torchvision.datasets import FashionMNIST
import pytorch_lightning as pl
import torchmetrics

In [2]:
print(torch.cuda.is_available())

True


In [3]:
class ResNet18(LightningModule):
    def __init__(self):
        super().__init__()
        self.model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False, num_classes=10)
        self.model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.accuracy = torchmetrics.Accuracy()
        self.train_acc = torchmetrics.Accuracy()
    def forward(self, x):
        return self.model(x)
    def validation_step(self, batch, batch_idx):
        x, y = batch
        preds = self(x)
        loss = F.cross_entropy(preds, y)
        acc = self.accuracy(preds, y)
        self.log("val_loss", loss)
        self.log('val_acc_step', acc)
        return loss
    def training_step(self, batch, batch_nb):
        x, y = batch
        preds = self(x)
        loss = F.cross_entropy(preds, y)
        acc = self.train_acc(preds, y)
        self.log('train_acc_step', acc)
        return loss
    def test_step(self, batch, batch_nb):
        x, y = batch
        loss = F.cross_entropy(self(x), y)
        return loss
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.0009)

In [4]:
model = ResNet18()
print(model)

Using cache found in /home/dsingh/.cache/torch/hub/pytorch_vision_v0.10.0


ResNet18(
  (model): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runn

In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torch
from pytorch_lightning import LightningModule, Trainer
from torch import nn
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset, random_split
from torchmetrics import Accuracy
from torchvision import transforms
from torchvision.datasets import FashionMNIST

train_loader = DataLoader(FashionMNIST('fashionmnist/', train=True, download=True, transform=transforms.ToTensor()), batch_size=64, num_workers=8)
test_loader = DataLoader(FashionMNIST('fashionmnist/', train=False, download=True, transform=transforms.ToTensor()), batch_size=64, num_workers=8)
print(len(train_loader), len(test_loader))
trainer = pl.Trainer(max_epochs=5,accelerator="gpu")

trainer.fit(model, train_loader, test_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


938 157


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type     | Params
---------------------------------------
0 | model     | ResNet   | 11.2 M
1 | accuracy  | Accuracy | 0     
2 | train_acc | Accuracy | 0     
---------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.701    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

In [ ]:
from tqdm import tqdm
accuracy = torchmetrics.Accuracy()
i = 0 
s = 0
with torch.no_grad():
    for item in tqdm(test_loader):
        preds = F.softmax(model(item[0]), dim=1)
        pred = torch.argmax(preds, dim=1)
        labels = item[1]
        s += accuracy(pred, labels).item()
        i += 1
print(s / i)